### Flat Field Correction - Preserves colors
What the LAB version does

#### Convert to CIELAB where L ≈ perceptual lightness, A/B ≈ chroma.
- Estimate shading on L only, compute a gain map, and apply it only to L.
- Leave A/B unchanged, so hues/chroma stay stable; then convert back to BGR.
#### Implications:
- Much better hue preservation (since A/B are untouched).
- More perceptually uniform correction (flatten illumination without “color drifting”).

We can clip the gain map to avoid blown highlights (np.clip(gain, 0.5, 2.0)), which is harder to control in the RGB min–max approach.

In [2]:
import cv2
import numpy as np
import os

# --- config ---
IMG_PATH = "./images/ncr_8338d.jpg"   # <-- change if needed
SIGMA = 101                                      # large blur for background (try 51..201)
EPS = 1e-6

# output path in same folder with "_corrected" suffix
root, ext = os.path.splitext(IMG_PATH)
OUT_PATH = f"{root}_corrected.jpg"

# --- load ---
bgr = cv2.imread(IMG_PATH, cv2.IMREAD_COLOR)
assert bgr is not None, f"Could not open {IMG_PATH}"

# --- convert to LAB (better for illumination correction) ---
lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB).astype(np.float32)
L, A, B = cv2.split(lab)  # L in [0..255]

# --- estimate low‑frequency illumination on L with a big Gaussian blur ---
# (use (0,0) kernel and SIGMA to let OpenCV pick proper kernel size)
illum = cv2.GaussianBlur(L, (0, 0), sigmaX=SIGMA, sigmaY=SIGMA)

# --- compute gain map to flatten lighting ---
# scale so that the average lightness is preserved
mean_L = np.mean(L)
gain = (mean_L / (illum + EPS))

# optional clipping of gain to avoid extreme boosts:
gain = np.clip(gain, 0.5, 2.0)

# --- apply gain to L only; keep A/B (color) unchanged ---
L_corr = np.clip(L * gain, 0, 255).astype(np.float32)

# --- merge & convert back to BGR ---
lab_corr = cv2.merge([L_corr, A, B]).astype(np.uint8)
bgr_corr = cv2.cvtColor(lab_corr, cv2.COLOR_LAB2BGR)

# (optional) gentle denoise to remove tiny speckle after correction
# bgr_corr = cv2.fastNlMeansDenoisingColored(bgr_corr, None, 3, 3, 7, 21)

cv2.imwrite(OUT_PATH, bgr_corr)
print("Saved:", OUT_PATH)


Saved: ./images/ncr_8338d_corrected.jpg
